In [ ]:
!pip install hmmlearn
!pip install pandas
from hmmlearn import hmm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dimensiunea gridului
dimensiune_grid = (10, 10)

# Lista de culori predefinite
culori = [
    "red", "blue", "green", "yellow",
    "purple", "orange", "pink", "cyan",
    "brown", "lime"
]

# Citirea gridului
df = pd.read_csv('/grid_culori.csv',header=None)
grid_culori = df.to_numpy

# Generarea secvenței de culori observate
observatii = ["red", "red", "lime", "yellow", "blue"]

# Mapare culori -> indecși
culoare_to_idx = {culoare: idx for idx, culoare in enumerate(culori)}
idx_to_culoare = {idx: culoare for culoare, idx in culoare_to_idx.items()}

# Transformăm secvența de observații în indecși
observatii_idx = [culoare_to_idx[c] for c in observatii]

# Definim stările ascunse ca fiind toate pozițiile din grid (100 de stări)
numar_stari = dimensiune_grid[0] * dimensiune_grid[1]
stari_ascunse = [(i, j) for i in range(dimensiune_grid[0]) for j in range(dimensiune_grid[1])]
stare_to_idx = {stare: idx for idx, stare in enumerate(stari_ascunse)}
idx_to_stare = {idx: stare for stare, idx in stare_to_idx.items()}

# Matrice de tranziție
transitions = np.zeros((numar_stari, numar_stari))
for i, j in stari_ascunse:
    stare_curenta = stare_to_idx[(i, j)]
    vecini = [
        (i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)  # sus, jos, stânga, dreapta
    ]
    vecini_valizi = [stare_to_idx[(x, y)] for x, y in vecini if 0 <= x < 10 and 0 <= y < 10]
    for vecin in vecini_valizi:
        if vecin == stare_curenta:
            transitions[stare_curenta, vecin] = 0.25
        else:
            transitions[stare_curenta, vecin] = 0.75 / (len(vecini_valizi) - 1)

# Matrice de emisie
emissions = np.zeros((numar_stari, len(culori)))
for idx, (i, j) in enumerate(stari_ascunse):
    culoare = grid_culori[i, j]
    if culoare in culoare_to_idx:
        emissions[idx, culoare_to_idx[culoare]] = 1.0
emissions /= emissions.sum(axis=1, keepdims=True)

start_probability = np.full(numar_stari, 1 / numar_stari)

# Modelul HMM
model = hmm.MultinomialHMM(n_components=numar_stari, n_iter=100)
model.startprob_ = start_probability
model.transmat_ = transitions
model.emissionprob_ = emissions

# Rulăm algoritmul Viterbi pentru secvența de observații
observatii_idx = np.array(observatii_idx).reshape(-1, 1)
logprob, secventa_stari = model.decode(observatii_idx, algorithm="viterbi")


# Convertim secvența de stări în poziții din grid
drum = [idx_to_stare[idx] for idx in secventa_stari]

# Vizualizăm drumul pe grid
fig, ax = plt.subplots(figsize=(8, 8))
for i in range(dimensiune_grid[0]):
    for j in range(dimensiune_grid[1]):
        culoare = grid_culori[i, j]
        ax.add_patch(plt.Rectangle((j, dimensiune_grid[0] - i - 1), 1, 1, color=culoare))
        ax.text(j + 0.5, dimensiune_grid[0] - i - 0.5, culoare,
                color="white", ha="center", va="center", fontsize=8, fontweight="bold")

# Evidențiem drumul rezultat
for idx, (i, j) in enumerate(drum):
    ax.add_patch(plt.Circle((j + 0.5, dimensiune_grid[0] - i - 0.5), 0.3, color="black", alpha=0.7))
    ax.text(j + 0.5, dimensiune_grid[0] - i - 0.5, str(idx + 1),
            color="white", ha="center", va="center", fontsize=10, fontweight="bold")

# Setări axă
ax.set_xlim(0, dimensiune_grid[1])
ax.set_ylim(0, dimensiune_grid[0])
ax.set_xticks(range(dimensiune_grid[1]))
ax.set_yticks(range(dimensiune_grid[0]))
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(visible=True, color="black", linewidth=0.5)
ax.set_aspect("equal")
plt.title("Drumul rezultat al stărilor ascunse", fontsize=14)
plt.show()

In [3]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

nr_aruncari=10
observatii=['s','b','b','b','s','b','s','b','b','b']

alpha_prior = 1
beta_prior = 1

with pm.Model() as model:


    beta_prior = pm.Uniform("beta", lower=1, upper=1)

    likelihood = pm.Normal("likelihood", mu=np.mean(observatii), sigma=1/beta_prior, observed=observatii)

    trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42)

az.plot_posterior(trace, var_names=["mu"], hdi_prob=0.95)
plt.title("Distributia a posteriori pentru mu")
plt.show()

# Rezultate statistice
print(az.summary(trace, var_names=["mu", "sigma"]))

SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'beta_interval__': array(nan)}

Logp initial evaluation results:
{'beta': nan}
You can call `model.debug()` for more details.

In [4]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

nr_aruncari=10
observatii=['s','s','b','s','b','b','s','b','b','s']

alpha_prior = 1
beta_prior = 1

prior_pdf = stats.gamma.pdf(observatii, a=alpha_prior, scale=1/beta_prior)

# Normalizam distributia a posteriori
posterior_pdf = unnormalized_posterior / np.trapz(unnormalized_posterior, observatii)

# Vizualizarea distributiei a posteriori
plt.figure(figsize=(10, 6))
plt.plot(observatii, posterior_pdf, label='Distribuția a posteriori a lui λ')
plt.ylabel('Densitatea de probabilitate')
plt.legend()
plt.grid(True)
plt.show()

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U1'), dtype('int64')) -> None